In [1]:
import numpy as np
import pandas as pd

In [2]:
def to_dict(line):
    line = line.split()
    line_dict = {}
    line_dict['ix'], line_dict['iy'], line_dict['icloudy'], line_dict['icol'], line_dict['irow'], \
    line_dict['x'], line_dict['y'], line_dict['MASL'], line_dict['land_percent'], line_dict['nwl'] = line[:10]
    
    wl_num = int(line_dict['nwl'])
    
    line_dict['wl'] = line[10:10 + wl_num]
    line_dict['nip'] = line[10 + wl_num: 10 + 2 * wl_num]
    
    chanel_num = sum([int(el) for el in line_dict['nip']])
    
    
    last = 10 + 2 * int(line_dict['nwl']) + chanel_num
    line_dict['meas_type'] = line[22: last]
    
    line_dict['nbvm'] = line[last: last + chanel_num]
    last = last + chanel_num
    
    
    meas_num = sum([int(el) for el in line_dict['nbvm']])

    line_dict['sza'] = line[last: last + wl_num]
    last = last + wl_num

    line_dict['vza'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['raa'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['meas'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['ifcov'] = line[last: last + chanel_num]
    last = last + chanel_num 

    line_dict['ifmp'] = line[last: last + chanel_num]
    last = last + chanel_num 
    return line_dict

In [3]:
def get_data(line_dict):
    wl_num = int(line_dict['nwl'])
    chanel_num = sum([int(el) for el in line_dict['nip']])
    meas_num = sum([int(el) for el in line_dict['nbvm']])
    
    def mult(x):
        if type(x) == list:

            if len(x) == wl_num:
                l = [[a]*int(b) for a,b in zip(x,line_dict['nip'])]
                x = [item for sublist in l for item in sublist]

            if len(x) == chanel_num:
                l = [[a]*int(b) for a,b in zip(x,line_dict['nbvm'])]
                x =  [item for sublist in l for item in sublist]

            return x
        return [x] * meas_num

    t = pd.Series(line_dict).apply(mult)
    return pd.DataFrame(t.values.tolist(), index = t.index).T

In [14]:
f = open("/home/ihor/work_space/aerosol_data_processing/example_polder_true.sdat", "r")
f_bani = open("/home/ihor/work_space/aerosol_data_processing/GRASP data simulation/Scene descriptions for Banizounbou and Mongu sites from 5 bin POLDER retrievals/Banizoumbou_080101_080331_2x2.dat", "r")

text = f.readlines() 
text_bani = f_bani.readlines() 

In [23]:
temp = pd.Series(text)
temp[temp.apply(len) == 103].head()

3       4   2008-06-14T14:49:28Z  705000.00   0   0 ...
10      4   2008-06-16T14:37:07Z  705000.00   0   0 ...
17      4   2008-06-18T14:24:47Z  705000.00   0   0 ...
24      4   2008-06-20T14:12:28Z  705000.00   0   0 ...
31      4   2008-06-23T14:43:18Z  705000.00   0   0 ...
dtype: object

In [24]:
temp = pd.Series(text_bani)
temp[temp.apply(len) == 71].head()

3     4 2008-01-04T13:15:01Z 705000 0 0 : NPIXELS TI...
9     4 2008-01-06T13:02:42Z 705000 0 0 : NPIXELS TI...
15    4 2008-01-08T12:50:22Z 705000 0 0 : NPIXELS TI...
21    4 2008-01-11T13:21:20Z 705000 0 0 : NPIXELS TI...
27    4 2008-01-13T13:09:03Z 705000 0 0 : NPIXELS TI...
dtype: object

In [25]:
data_list = []
for line in text_bani:
    if len(line) == 71:
        cell_header = {}
        cell_header['NPIXELS'], cell_header['TIMESTAMP'], cell_header['HEIGHT_OBS'], \
        cell_header['NSURF'], cell_header['IFGAS'] = line.split()[:5]
    elif len(line) > 200:
        line_dict = to_dict(line)
        line_dict.update(cell_header)
        data = get_data(line_dict)
        data_list.append(data)
        

In [26]:
data = pd.concat(data_list)

In [28]:
data.to_csv('polder_Banizounbou_normalize_data.csv', index=False)

In [29]:
line_dict = to_dict(line)